# Theano 循环：scan（详解）

In [1]:
import theano
import theano.tensor as T
import numpy as np

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

Using gpu device 1: Tesla K10.G2.8GB (CNMeM is disabled)


`theano` 中可以使用 `scan` 进行循环，常用的 `map` 和 `reduce` 操作都可以看成是 `scan` 的特例。 

`scan` 通常作用在一个序列上，每次处理一个输入，并输出一个结果。

`sum(x)` 函数可以看成是 `z + x(i)` 函数在给定 `z = 0` 的情况下，对 `x` 的一个 `scan`。

通常我们可以将一个 `for` 循环表示成一个 `scan` 操作，其好处如下：

- 迭代次数成为符号图结构的一部分
- 最小化 GPU 数据传递
- 序列化梯度计算
- 速度比 `for` 稍微快一些
- 降低内存使用

## scan 的使用

函数的用法如下：

    theano.scan(fn, 
                sequences=None, 
                outputs_info=None, 
                non_sequences=None, 
                n_steps=None, 
                truncate_gradient=-1, 
                go_backwards=False, 
                mode=None, 
                name=None, 
                profile=False, 
                allow_gc=None, 
                strict=False)
                
主要参数的含义：

- `fn`
    - 一步 `scan` 所进行的操作
- `sequences`
    - 输入的序列
- `outputs_info`
    - 前一步输出结果的初始状态
- `non_sequences`
    - 非序列参数
- `n_steps`
    - 迭代步数
- `go_backwards`
    - 是否从后向前遍历
    
输出为一个元组 `(outputs, updates)`：

- `output`
    - 从初始状态开始，每一步 `fn` 的输出结果
- `updates`
    - 一个字典，用来记录 `scan` 过程中用到的内部共享变量，构造函数的时候，需要将这个变量当作 `updates` 的参数传入。

##  scan 和 map

这里实现一个简单的 `map` 操作，将向量 $\mathbf x$ 中的所有元素变成原来的两倍：

```python
map(lambda t: t * 2, x)
```

In [2]:
x = T.vector()

result, updates = theano.scan(fn = lambda t: t * 2,
                              sequences = x)
x_double_scan = theano.function([x], result, updates = updates)

print x_double_scan(range(10))

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


之前我们说到，`theano` 中的 `map` 是 `scan` 的一个特例，因此 `theano.map` 的用法其实跟 `theano.scan` 十分类似。

由于不需要考虑前一步的输出结果，所以 `theano.map` 的参数中没有 `outputs_info` 这一部分。

我们用 `theano.map` 实现相同的效果：

In [3]:
result, updates = theano.map(fn = lambda t: t * 2,
                              sequences = x)
x_double_map = theano.function([x], result, updates = updates)

print x_double_map(range(10))

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


## scan 和 reduce

这里一个简单的 `reduce` 操作，求和：

```python
reduce(lambda a, b: a + b, x)
```

In [4]:
result, updates = theano.scan(fn = lambda t, v: t + v,
                              sequences = x,
                              outputs_info = 0.)

# 因为每一步的输出值都会被记录到最后的 result 中，所以最后的和是 result 的最后一个元素。
x_sum_scan = theano.function([x], result[-1], updates=updates)

print x_sum_scan(range(10))

45.0


`theano.reduce` 也是 `scan` 的一个特例，使用 `theano.reduce` 实现相同的效果： 

In [5]:
result, updates = theano.reduce(fn = lambda t, v: t + v,
                              sequences = x,
                              outputs_info = 0.)

x_sum_reduce = theano.function([x], result, updates=updates)

print x_sum_reduce(range(10))

45.0


`reduce` 与 `scan` 不同的地方在于，`result` 包含的内容并不是每次输出的结果，而是最后一次输出的结果。